<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Find the distance between smoker and not

In [32]:
import cv2
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple
import math
import os
import json

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
@dataclass
class Person:
    x1: int
    y1: int
    x2: int
    y2: int
    is_smoking: bool
    confidence: float

In [35]:
def calculate_pixels_per_meter(image_width: int, original_width: int = 2048) -> float:
    """
    Calcola il fattore di scala basato sulle dimensioni originali dell'immagine
    Assumendo che nell'immagine originale 100 pixel = 1 metro
    """
    return (100 * image_width) / original_width

In [36]:
def calculate_center_point(person: Person) -> Tuple[float, float]:
    """Calcola il punto centrale di una persona"""
    center_x = (person.x1 + person.x2) / 2
    center_y = (person.y1 + person.y2) / 2
    return (center_x, center_y)

In [37]:
def calculate_distance(p1: Person, p2: Person, pixels_per_meter: float) -> float:
    """Calcola la distanza euclidea tra due persone"""
    c1 = calculate_center_point(p1)
    c2 = calculate_center_point(p2)

    pixel_distance = math.sqrt(
        (c2[0] - c1[0])**2 + (c2[1] - c1[1])**2
    )

    return pixel_distance / pixels_per_meter

In [38]:
def find_smoker_nonsmoker_distances(people: List[Person], pixels_per_meter: float) -> List[Tuple[Person, Person, float]]:
    """
    Trova tutte le distanze tra fumatori e non fumatori
    """
    smokers = [p for p in people if p.is_smoking]
    non_smokers = [p for p in people if not p.is_smoking]
    distances = []

    for smoker in smokers:
        for non_smoker in non_smokers:
            distance = calculate_distance(smoker, non_smoker, pixels_per_meter)
            distances.append((smoker, non_smoker, distance))

    return distances

In [39]:
def load_detections_from_json(json_path: str) -> List[Person]:
    """
    Carica le detection dal file JSON e le converte in oggetti Person
    """
    with open(json_path, 'r') as f:
        data = json.load(f)

    people = []
    for detection in data['detections']:
        # Classe 2 rappresenta il fumatore, 1 il non fumatore
        is_smoking = detection['class'] == 2
        is_not_smoking = detection['class'] == 1
        bbox = detection['bbox']
        people.append(Person(
            x1=int(bbox[0]),
            y1=int(bbox[1]),
            x2=int(bbox[2]),
            y2=int(bbox[3]),
            is_smoking=is_smoking,
            confidence=detection['confidence']
        ))

    return people

In [40]:
def process_and_save_image(image_path: str, people: List[Person], output_dir: str) -> None:
    """
    Processa un'immagine disegnando le distanze tra i centri delle bounding box
    """
    os.makedirs(output_dir, exist_ok=True)

    # Carica l'immagine
    image = cv2.imread(image_path)
    if image is None:
        print(f"Errore nel caricamento dell'immagine: {image_path}")
        return

    # Calcola il fattore di scala basato sulla larghezza dell'immagine
    pixels_per_meter = calculate_pixels_per_meter(image.shape[1])

    # Definizione colori
    YELLOW = (0, 255, 255)  # BGR per giallo (centri)
    BROWN = (42, 42, 165)   # BGR per marrone (linee distanza)
    RED = (0, 0, 255)       # BGR per rosso (fumatori)
    BLUE = (255, 0, 0)      # BGR per blu (non fumatori)

    # Disegna le bounding box e i centri
    for person in people:
        # Colore basato sul tipo di persona
        color = RED if person.is_smoking else BLUE

        # Disegna la bounding box
        cv2.rectangle(image,
                     (int(person.x1), int(person.y1)),
                     (int(person.x2), int(person.y2)),
                     color, 2)

        # Disegna il punto centrale
        center = calculate_center_point(person)
        cv2.circle(image, (int(center[0]), int(center[1])), 3, YELLOW, -1)

    # Calcola e disegna le distanze
    distances = find_smoker_nonsmoker_distances(people, pixels_per_meter)
    for smoker, non_smoker, distance in distances:
        s_center = calculate_center_point(smoker)
        ns_center = calculate_center_point(non_smoker)

        # Disegna la linea tra i centri
        cv2.line(image,
                (int(s_center[0]), int(s_center[1])),
                (int(ns_center[0]), int(ns_center[1])),
                BROWN, 2)

        # Aggiungi il testo della distanza
        mid_point = ((s_center[0] + ns_center[0])//2, (s_center[1] + ns_center[1])//2)
        cv2.putText(image,
                   f"{distance:.2f}m",
                   (int(mid_point[0]), int(mid_point[1])),
                   cv2.FONT_HERSHEY_SIMPLEX,
                   0.7,
                   BROWN,
                   2)

    output_path = os.path.join(output_dir, f"distances_{os.path.basename(image_path)}")
    cv2.imwrite(output_path, image)
    print(f"Distanze calcolate usando {pixels_per_meter:.2f} pixels/metro")

In [41]:
def main():
    base_dir = '/content/drive/MyDrive/test_trained_person'
    output_dir = '/content/drive/MyDrive/distance_img_process'

    # Processa tutte le immagini nella cartella
    images_dir = os.path.join(base_dir, 'images')
    coordinates_dir = os.path.join(base_dir, 'coordinates')

    for filename in os.listdir(images_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Costruisci i percorsi dei file
            image_path = os.path.join(images_dir, filename)
            json_name = f"{os.path.splitext(filename)[0]}.json"
            json_path = os.path.join(coordinates_dir, json_name)

            if os.path.exists(json_path):
                try:
                    # Carica le detection dal JSON
                    people = load_detections_from_json(json_path)
                    # Processa l'immagine
                    process_and_save_image(image_path, people, output_dir)
                    print(f"Processata immagine: {filename}")
                except Exception as e:
                    print(f"Errore nel processare {filename}: {str(e)}")
            else:
                print(f"File JSON non trovato per {filename}")

if __name__ == "__main__":
    main()

Errore nel processare trained_40.jpg: Person.__init__() got an unexpected keyword argument 'is_not_smoking'
Errore nel processare trained_41.jpg: Person.__init__() got an unexpected keyword argument 'is_not_smoking'
Errore nel processare trained_42.jpg: Person.__init__() got an unexpected keyword argument 'is_not_smoking'
Errore nel processare trained_43.jpg: Person.__init__() got an unexpected keyword argument 'is_not_smoking'
Errore nel processare trained_44.jpg: Person.__init__() got an unexpected keyword argument 'is_not_smoking'
Errore nel processare trained_45.jpg: Person.__init__() got an unexpected keyword argument 'is_not_smoking'
Errore nel processare trained_46.jpg: Person.__init__() got an unexpected keyword argument 'is_not_smoking'
Errore nel processare trained_47.jpg: Person.__init__() got an unexpected keyword argument 'is_not_smoking'
Errore nel processare trained_48.jpg: Person.__init__() got an unexpected keyword argument 'is_not_smoking'
Errore nel processare traine